In [ ]:
import os,sys
os.environ["CUDA_VISIBLE_DEVICES"]="1"
sys.path.append('..')
sys.path.append('.')
if not os.path.exists('VGDFR'):
    os.chdir('../')
# print current work dir
print(os.getcwd())
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import torch
from hyvideo.utils.file_utils import save_videos_grid
from pathlib import Path
from datetime import datetime
import time
from hyvideo.config import *
from VGDFR.hunyuan_vgdfr import VGDFRHunyuanVideoSampler


In [ ]:
string_args="""--video-size 544 960 --video-length 65 --infer-steps 50 --prompt cat. --flow-reverse --use-cpu-offload --save-path ./results --seed 3"""
string_args=string_args.split(" ")
print(string_args)

def parse_args_new(namespace=None,string_args=None):
    parser = argparse.ArgumentParser(description="HunyuanVideo inference script")

    parser = add_network_args(parser)
    parser = add_extra_models_args(parser)
    parser = add_denoise_schedule_args(parser)
    parser = add_inference_args(parser)
    parser = add_parallel_args(parser)

    args = parser.parse_args(string_args,namespace=namespace)
    args = sanity_check_args(args)

    return args

args = parse_args_new(string_args=string_args)
print(args)


In [ ]:
models_root_path = Path(args.model_base)
hunyuan_video_sampler = VGDFRHunyuanVideoSampler.from_pretrained(models_root_path, args=args)

In [ ]:
hunyuan_video_sampler.pipeline.schedule_mode = "compress_ratio"
seed = 3
args.infer_steps = 50
width, height = 960, 544
width, height = 500, 344
video_length = 97
prompt="A spirited individual rides a vintage bicycle along a sunlit, tree-lined path, wearing a casual outfit of a white t-shirt, denim shorts, and sneakers. The scene captures the golden hour, with sunlight filtering through the leaves, casting dappled shadows on the ground. The rider's hair flows freely in the breeze, and a joyful smile lights up their face. As they pedal, the camera zooms in to reveal the intricate details of the bike's design, including its classic handlebars and shiny bell. The background features a serene park with blooming flowers and a distant lake, enhancing the sense of freedom and tranquility."
for before_compression_steps in [5, 10]:
    hunyuan_video_sampler.pipeline.before_compression_steps = before_compression_steps
    for compress_ratio in [0.5, 0.6, 0.7, 0.8, 0.9]:
        hunyuan_video_sampler.pipeline.compress_ratio = compress_ratio

        samples = hunyuan_video_sampler.predict(
            prompt=prompt,
            height=height,
            width=width,
            video_length=video_length,
            seed=seed,
            negative_prompt=args.neg_prompt,
            infer_steps=args.infer_steps,
            guidance_scale=args.cfg_scale,
            num_videos_per_prompt=args.num_videos,
            flow_shift=args.flow_shift,
            batch_size=args.batch_size,
            embedded_guidance_scale=args.embedded_cfg_scale,
        )["samples"]
        save_path = args.save_path
        # log_dlfr_t = hunyuan_video_sampler.pipeline.log_dlfr_t
        # Save samples
        if "LOCAL_RANK" not in os.environ or int(os.environ["LOCAL_RANK"]) == 0:
            for i, sample in enumerate(samples):
                sample = samples[i].unsqueeze(0)
                time_flag = datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d-%H:%M:%S")
                file_name = f"raw_seed{seed}_{prompt[:100].replace('/','')}"
                raw_save_path = (
                    f"{save_path}/example/cr_{compress_ratio}_k{before_compression_steps}/{file_name}.mp4"
                )
                save_videos_grid(sample, raw_save_path, fps=12)
                torch.save(
                    sample,
                    f"{save_path}/example/cr_{compress_ratio}_k{before_compression_steps}/{file_name}.pt",
                )


100%|██████████| 45/45 [01:45<00:00,  2.34s/it]


Decoding tile shape: torch.Size([1, 16, 17, 44, 64]), from 0 to 17
Decoding tile shape: torch.Size([1, 16, 9, 44, 64]), from 12 to 29


2025-04-20 23:40:18.255 | INFO     | hyvideo.inference:predict:669 - Success, time: 148.435142993927
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2025-04-20 23:40:19.742 | INFO     | hyvideo.inference:predict:580 - Input (height, width, video_length) = (344, 500, 97)
2025-04-20 23:40:19.750 | DEBUG    | hyvideo.inference:predict:640 - 
                        height: 352
                         width: 512
                  video_length: 97
                        prompt: ["A spirited individual rides a vintage bicycle along a sunlit, tree-lined path, wearing a casual outfit of a white t-shirt, denim shorts, and sneakers. The scene captures the golden hour, with sunlight filtering through the leaves, casting dap

Run VGDFR Compression Module...
Decoding tile shape: torch.Size([1, 16, 17, 44, 64]), from 0 to 17
Decoding tile shape: torch.Size([1, 16, 13, 44, 64]), from 12 to 29
Decoding tile shape: torch.Size([1, 16, 1, 44, 64]), from 24 to 41
tensor(1., device='cuda:0', dtype=torch.float16) tensor(0., device='cuda:0', dtype=torch.float16)
Using compress ratio schedule with compress_ratio=0.9 and k=5
Now status: 0.5463917525773195 < 0.9, left_similarity_threshold: 0.5
Now status: 0.711340206185567 < 0.9, left_similarity_threshold: 0.745
Now status: 1.0 > 0.9, right_similarity_threshold: 0.8674999999999999
Now status: 0.8762886597938144 < 0.9, left_similarity_threshold: 0.8062499999999999
Now status: 0.9175257731958762 > 0.9, right_similarity_threshold: 0.8368749999999999
Now status: 0.8762886597938144 < 0.9, left_similarity_threshold: 0.8215625
Now status: 0.9175257731958762 > 0.9, right_similarity_threshold: 0.8292187499999999
Now status: 0.9175257731958762 > 0.9, right_similarity_threshold: 0.

  9%|▉         | 4/45 [00:10<01:49,  2.66s/it]

In [ ]:
hunyuan_video_sampler.pipeline.schedule_mode = "similarity_threshold"
seed = 3
args.infer_steps = 50
width, height = 960, 544
width, height = 500, 344
video_length = 97
prompt="A spirited individual rides a vintage bicycle along a sunlit, tree-lined path, wearing a casual outfit of a white t-shirt, denim shorts, and sneakers. The scene captures the golden hour, with sunlight filtering through the leaves, casting dappled shadows on the ground. The rider's hair flows freely in the breeze, and a joyful smile lights up their face. As they pedal, the camera zooms in to reveal the intricate details of the bike's design, including its classic handlebars and shiny bell. The background features a serene park with blooming flowers and a distant lake, enhancing the sense of freedom and tranquility."
for before_compression_steps in [5, 10, 15]:
    hunyuan_video_sampler.pipeline.before_compression_steps = before_compression_steps
    for similarity_threshold in [0.6, 0.7, 0.8, 0.9]:
        hunyuan_video_sampler.pipeline.similarity_threshold = similarity_threshold

        samples = hunyuan_video_sampler.predict(
            prompt=prompt,
            height=height,
            width=width,
            video_length=video_length,
            seed=seed,
            negative_prompt=args.neg_prompt,
            infer_steps=args.infer_steps,
            guidance_scale=args.cfg_scale,
            num_videos_per_prompt=args.num_videos,
            flow_shift=args.flow_shift,
            batch_size=args.batch_size,
            embedded_guidance_scale=args.embedded_cfg_scale,
        )["samples"]
        save_path = args.save_path
        # log_dlfr_t = hunyuan_video_sampler.pipeline.log_dlfr_t
        # Save samples
        if "LOCAL_RANK" not in os.environ or int(os.environ["LOCAL_RANK"]) == 0:
            for i, sample in enumerate(samples):
                sample = samples[i].unsqueeze(0)
                time_flag = datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d-%H:%M:%S")
                file_name = f"raw_seed{seed}_{prompt[:100].replace('/','')}"
                raw_save_path = (
                    f"{save_path}/example/th_{similarity_threshold}_k{before_compression_steps}/{file_name}.mp4"
                )
                save_videos_grid(sample, raw_save_path, fps=12)
                torch.save(
                    sample,
                    f"{save_path}/example/th_{similarity_threshold}_k{before_compression_steps}/{file_name}.pt",
                )
